# 5강) BERT를 활용한 Dense Passage Retrieval 실습

### Requirements

In [1]:
import torch
torch.set_printoptions(threshold=10_000)

In [2]:
# !pip install datasets
# !pip install transformers

## 데이터셋 로딩


KorQuAD train 데이터셋을 학습 데이터로 활용

In [1]:
from datasets import load_dataset, load_from_disk

# dataset = load_dataset("squad_kor_v1")
dataset = load_from_disk('../../data/train_dataset')

## 토크나이저 준비 - Huggingface 제공 tokenizer 이용

BERT를 encoder로 사용하므로, hugginface에서 제공하는 "bert-base-multilingual-cased" tokenizer를 활용

In [2]:
from transformers import AutoTokenizer
import numpy as np

# model_checkpoint = "bert-base-multilingual-cased"
# model_checkpoint = "klue/roberta-base"
model_checkpoint = "klue/bert-base"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)


In [3]:
tokenizer

PreTrainedTokenizerFast(name_or_path='klue/bert-base', vocab_size=32000, model_max_len=512, is_fast=True, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [4]:
tokenized_input = tokenizer(dataset['train'][0]['context'], padding="max_length", truncation=True)
tokenizer.decode(tokenized_input['input_ids'])

'[CLS] 미국 상의원 또는 미국 상원 ( United States Senate ) 은 양원제인 미국 의회의 상원이다. [UNK] n [UNK] n미국 부통령이 상원의장이 된다. 각 주당 2명의 상원의원이 선출되어 100명의 상원의원으로 구성되어 있다. 임기는 6년이며, 2년마다 50개주 중 1 / 3씩 상원의원을 새로 선출하여 연방에 보낸다. [UNK] n [UNK] n미국 상원은 미국 하원과는 다르게 미국 대통령을 수반으로 하는 미국 연방 행정부에 각종 동의를 하는 기관이다. 하원이 세금과 경제에 대한 권한, 대통령을 포함한 대다수의 공무원을 파면할 권한을 갖고 있는 국민을 대표하는 기관인 반면 상원은 미국의 주를 대표한다. 즉 캘리포니아주, 일리노이주 같이 주 정부와 주 의회를 대표하는 기관이다. 그로 인하여 군대의 파병, 관료의 임명에 대한 동의, 외국 조약에 대한 승인 등 신속을 요하는 권한은 모두 상원에게만 있다. 그리고 하원에 대한 견제 역할 ( 하원의 법안을 거부할 권한 등 ) 을 담당한다. 2년의 임기로 인하여 급진적일 수밖에 없는 하원은 지나치게 급진적인 법안을 만들기 쉽다. 대표적인 예로 건강보험 개혁 당시 하원이 미국 연방 행정부에게 퍼블릭 옵션 ( 공공건강보험기관 ) 의 조항이 있는 반면 상원의 경우 하원안이 지나치게 세금이 많이 든다는 이유로 퍼블릭 옵션 조항을 제외하고 비영리건강보험기관이나 보험회사가 담당하도록 한 것이다. 이 경우처럼 상원은 하원이나 내각책임제가 빠지기 쉬운 국가들의 국회처럼 걸핏하면 발생하는 의회의 비정상적인 사태를 방지하는 기관이다. 상원은 급박한 처리사항의 경우가 아니면 법안을 먼저 내는 경우가 드물고 하원이 만든 법안을 수정하여 다시 하원에 되돌려보낸다. 이러한 방식으로 단원제가 빠지기 쉬운 함정을 미리 방지하는 것이다. 날짜 = 2017 - 02 - 05 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [

## Dense encoder (BERT) 학습 시키기

HuggingFace BERT를 활용하여 question encoder, passage encoder 학습

In [5]:
from tqdm import tqdm, trange
import argparse
import random
import torch
import torch.nn.functional as F
from transformers import BertModel, BertPreTrainedModel, AdamW, TrainingArguments, get_linear_schedule_with_warmup

torch.manual_seed(2021)
torch.cuda.manual_seed(2021)
np.random.seed(2021)
random.seed(2021)

1) Training Dataset 준비하기 (question, passage pairs)

---



In [6]:
# Use subset (128 example) of original training dataset 
# sample_idx = np.random.choice(range(len(dataset['train'])), 128)
# training_dataset = dataset['train'][sample_idx]

training_dataset = dataset['train']

In [7]:
from torch.utils.data import (DataLoader, RandomSampler, TensorDataset)

# q_seqs = tokenizer(training_dataset['question'], padding="max_length", truncation=True, return_tensors='pt', return_token_type_ids=False)
# p_seqs = tokenizer(training_dataset['context'], padding="max_length", truncation=True, return_tensors='pt', return_token_type_ids=False)
q_seqs = tokenizer(training_dataset['question'], padding="max_length", truncation=True, return_tensors='pt')
p_seqs = tokenizer(training_dataset['context'], padding="max_length", truncation=True, return_tensors='pt')


In [8]:
train_dataset = TensorDataset(p_seqs['input_ids'], p_seqs['attention_mask'], p_seqs['token_type_ids'],
                        q_seqs['input_ids'], q_seqs['attention_mask'], q_seqs['token_type_ids'],)

2) BERT encoder 학습시키기

BertEncoder 모델 정의 후, question encoder, passage encoder에 pre-trained weight 불러오기

In [9]:
from transformers import RobertaPreTrainedModel, RobertaModel

In [10]:
# class BertEncoder(RobertaPreTrainedModel):
class BertEncoder(BertPreTrainedModel):  
  def __init__(self, config):
    super(BertEncoder, self).__init__(config)

    # self.bert = RobertaModel(config)
    self.bert = BertModel(config)
    self.init_weights()
      
  def forward(self, input_ids, 
              attention_mask=None, token_type_ids=None): 
  
      outputs = self.bert(input_ids,
                          attention_mask=attention_mask,
                          token_type_ids=token_type_ids #roberta시 주석
                          )
      
      pooled_output = outputs[1]

      return pooled_output


In [11]:
# load pre-trained model on cuda (if available)
p_encoder = BertEncoder.from_pretrained(model_checkpoint)
q_encoder = BertEncoder.from_pretrained(model_checkpoint)

if torch.cuda.is_available():
  p_encoder.cuda()
  q_encoder.cuda()

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertEncoder: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at klue/bert-base were not used when initializing BertEncoder: ['cls.predictions.tr

In [12]:
p_encoder

BertEncoder(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)


In [27]:
for i in iter(p_encoder.named_parameters()):
    print(i)

('bert.embeddings.word_embeddings.weight', Parameter containing:
tensor([[-0.0117, -0.0294,  0.1734,  ..., -0.0210, -0.0633, -0.0330],
        [-0.0576,  0.0024, -0.0457,  ..., -0.0083, -0.0166,  0.0642],
        [-0.0072, -0.0268,  0.1987,  ..., -0.0231, -0.0464, -0.0337],
        ...,
        [-0.0064,  0.0004,  0.1504,  ..., -0.0410, -0.0617, -0.0576],
        [-0.0407,  0.0250,  0.1770,  ..., -0.0886, -0.0763, -0.0640],
        [-0.0664,  0.0054,  0.1411,  ..., -0.0169, -0.0700, -0.0348]],
       device='cuda:0', requires_grad=True))
('bert.embeddings.position_embeddings.weight', Parameter containing:
tensor([[-0.0399,  0.0102,  0.1688,  ..., -0.0537, -0.0648,  0.0007],
        [ 0.0039, -0.0208, -0.1180,  ..., -0.0177,  0.0192, -0.0031],
        [-0.0062, -0.0101, -0.0763,  ...,  0.0020,  0.0332, -0.0009],
        ...,
        [ 0.0215, -0.0112, -0.0466,  ..., -0.0027, -0.0181, -0.0260],
        [ 0.0271, -0.0192, -0.0434,  ..., -0.0036,  0.0283,  0.0156],
        [ 0.0802,  0.011

Train function 정의 후, 두개의 encoder fine-tuning 하기 (In-batch negative 활용) 


In [22]:
def train(args, dataset, p_model, q_model):
  no_decay = ['bias', 'LayerNorm.weight']
  optimizer_grouped_parameters = [
    {"params": [p for n, p in p_encoder.named_parameters() if not any(nd in n for nd in no_decay)], "weight_decay": args.weight_decay},
    {"params": [p for n, p in p_encoder.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
    {"params": [p for n, p in q_encoder.named_parameters() if not any(nd in n for nd in no_decay)], "weight_decay": args.weight_decay},
    {"params": [p for n, p in q_encoder.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0}
  ]

  optimizer = AdamW(
    optimizer_grouped_parameters,
    lr=args.learning_rate,
    eps=args.adam_epsilon
  )
  
  # Dataloader
  train_sampler = RandomSampler(dataset)
  train_dataloader = DataLoader(dataset, sampler=train_sampler, batch_size=args.per_device_train_batch_size)

# tt
  t_total = len(train_dataloader) // args.gradient_accumulation_steps * args.num_train_epochs
  scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=args.warmup_steps, num_training_steps=t_total)

  # Start training!
  global_step = 0
  
  p_model.zero_grad()
  q_model.zero_grad()
  torch.cuda.empty_cache()
  
  train_iterator = trange(int(args.num_train_epochs), desc="Epoch")

  for _ in train_iterator:
    epoch_iterator = tqdm(train_dataloader, desc="Iteration")

    for step, batch in enumerate(epoch_iterator):
      q_encoder.train()
      p_encoder.train()
      
      if torch.cuda.is_available():
        batch = tuple(t.cuda() for t in batch)

      p_inputs = {'input_ids': batch[0],
                  'attention_mask': batch[1],
                  'token_type_ids': batch[2] # roberta시 주석
                  }
      
      q_inputs = {'input_ids': batch[3],
                  'attention_mask': batch[4],
                  'token_type_ids': batch[5] # roberta시 주석
                  }
      
      p_outputs = p_model(**p_inputs)  # (batch_size, emb_dim)
      q_outputs = q_model(**q_inputs)  # (batch_size, emb_dim)


      # Calculate similarity score & loss
      sim_scores = torch.matmul(q_outputs, torch.transpose(p_outputs, 0, 1))  # (batch_size, emb_dim) x (emb_dim, batch_size) = (batch_size, batch_size)
      
      # print('q_outputs: ',q_outputs)
      # print('p_outputs: ',p_outputs)
      # print('sim_scores: ',sim_scores)
        
      # target: position of positive samples = diagonal element 
      targets = torch.arange(0, args.per_device_train_batch_size).long()
      if torch.cuda.is_available():
        targets = targets.to('cuda')

      sim_scores = F.log_softmax(sim_scores, dim=1)
      
      

      loss = F.nll_loss(sim_scores, targets)
      print(loss)

      loss.backward()
      optimizer.step()
      scheduler.step()
      q_model.zero_grad()
      p_model.zero_grad()
      global_step += 1
      
      torch.cuda.empty_cache()


    
  return p_model, q_model




In [23]:
args = TrainingArguments(
    output_dir="dense_retireval",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=5,
    weight_decay=0.01
)


In [24]:
from tqdm import tqdm

In [26]:
p_encoder, q_encoder = tqdm(train(args, train_dataset, p_encoder, q_encoder))

Iteration:   0%|          | 0/988 [00:00<?, ?it/s]

tensor(5.0809, device='cuda:0', grad_fn=<NllLossBackward>)



Iteration:   0%|          | 1/988 [00:00<09:14,  1.78it/s]

tensor(5.9605e-08, device='cuda:0', grad_fn=<NllLossBackward>)



Iteration:   0%|          | 2/988 [00:01<08:07,  2.02it/s]

...

tensor(0.0012, device='cuda:0', grad_fn=<NllLossBackward>)



Iteration:   3%|▎         | 32/988 [00:18<08:57,  1.78it/s]

tensor(1.9045, device='cuda:0', grad_fn=<NllLossBackward>)


Epoch:   0%|          | 0/5 [00:18<?, ?it/s]


KeyboardInterrupt: 

In [25]:
p_encoder.save_pretrained('encoders/p_encoder')
q_encoder.save_pretrained('encoders/q_encoder')

## Dense Embedding을 활용하여 passage retrieval 실습해보기

In [26]:

valid_corpus = list(set([example['context'] for example in dataset['validation']]))#[:10]
sample_idx = random.choice(range(len(dataset['validation'])))
query = dataset['validation'][sample_idx]['question']
ground_truth = dataset['validation'][sample_idx]['context']

if not ground_truth in valid_corpus:
  valid_corpus.append(ground_truth)

print(query)
print(ground_truth, '\n\n')

# valid_corpus

주이로부터 문흠의 가짜 항복 소식을 들은 인물은?
아버지 주환이 고위 관료였기 때문에, 낭(郞)에 서임되었다. 기도위가 되어, 주환의 병사를 대신 거느렸다\n\n적오 4년(241년), 오나라에서 병사를 크게 일으켜 위나라를 세 방향으로 치고 들어갔을 때, 주연을 따라 위나라의 번성을 공격했고, 여거와 함께 성의 외곽 방어를 깨트렸다. 주연 군은 사마의의 구원군을 보고 달아나다가 대패했으나, 주이는 이 공적으로 돌아와서는 편장군이 되었다\n\n위나라의 여강태수 문흠이 육안에 주둔하면서 수많은 요새를 구축하고, 오나라 사람들에게 모반·망명을 유도하여 오나라에 피해를 입히고 있었다. 주이는 몸소 수하 2천 명을 거느리고 문흠의 일곱 주둔지를 격파하여 수백 급을 베었고, 이 공적으로 양무장군으로 전임했다\n\n적오 13년(250년) 10월 , 당시 양주자사인 문흠의 투항 밀사를 받아 친히 문흠을 맞이해 달라는 청을 받았다. 그러나 이것이 거짓 투항임을 간파하고 대제 손권에게 아뢨으며, 손권이 받아들여 여거에게 2만 명을 거느리고 주이를 도와 함께 국경에서 문흠을 기다리게 했다. 문흠은 결국 투항하지 않았다\n\n건흥 원년(252년)에는 진남장군으로 전임했다. 이해 12월, 위나라에서 대제가 죽은 틈을 노리고 군사를 크게 일으켜 세 방향으로 오나라를 쳤으며, 제갈탄과 호준이 그 중 한 방향을 맡아 지난달에 오나라의 태부 제갈각이 쌓은 동흥의 두 성을 함몰하고자 했다. 주이는 이 싸움에서 수군을 거느리고 위나라의 부교를 쳐 무너뜨려 위나라 군사를 크게 무찔렀다\n\n태평 2년(258년) 6월 , 손침의 명령을 받고 호림(虎林)에서 출발하여 하구독 손일을 암습하려 했는데, 무창에 이르렀을 때 손일이 이를 알아채고 위나라로 달아났다. 7월에 하구에 이르렀다 가절을 받고 전부독(손량전에 따른 것으로, 주환전에서는 대도독)이 되어, 수춘에서 위나라에 반기를 들고 오나라에 도움을 구한 제갈탄을 도와 수춘성의 포위를 풀고자 출진했다. 처음에는 3만 명을 거느리고 안풍성에 주둔하며 

앞서 학습한 passage encoder, question encoder을 이용해 dense embedding 생성

In [27]:
def to_cuda(batch):
  return tuple(t.cuda() for t in batch)

In [34]:
with torch.no_grad():
  p_encoder.eval()
  q_encoder.eval()

  # q_seqs_val = tokenizer([query], padding="max_length", truncation=True, return_tensors='pt').to('cuda')
  q_seqs_val = tokenizer(['해바라기는 무슨 꽃일까?'], padding="max_length", truncation=True, return_tensors='pt').to('cuda')
  q_emb = q_encoder(**q_seqs_val).to('cpu')  #(num_query, emb_dim)

  p_embs = []
  for p in valid_corpus:
    p = tokenizer(p, padding="max_length", truncation=True, return_tensors='pt').to('cuda')
    p_emb = p_encoder(**p).to('cpu').numpy()
    p_embs.append(p_emb)

p_embs = torch.Tensor(p_embs).squeeze()  # (num_passage, emb_dim)

print(p_embs.size(), q_emb.size())

torch.Size([235, 768]) torch.Size([1, 768])


생성된 embedding에 dot product를 수행 => Document들의 similarity ranking을 구함

In [35]:
dot_prod_scores = torch.matmul(q_emb, torch.transpose(p_embs, 0, 1))
print(dot_prod_scores.size())

rank = torch.argsort(dot_prod_scores, dim=1, descending=True).squeeze()
print(dot_prod_scores)
print(rank)

torch.Size([1, 235])
tensor([[232.3053, 238.3758, 238.3100, 239.5861, 237.4889, 234.5634, 237.9858,
         232.1852, 239.5936, 236.0988, 239.5917, 226.3246, 235.2983, 237.6539,
         236.3513, 229.4015, 231.9859, 234.8343, 236.5229, 236.3486, 238.0967,
         232.0994, 235.6399, 233.9610, 236.2306, 232.0301, 235.5572, 226.9109,
         239.6329, 233.5367, 231.6475, 232.0976, 235.4974, 234.1947, 232.7057,
         235.0099, 239.1635, 239.5350, 229.5685, 238.0921, 242.5622, 244.0243,
         239.1308, 237.0562, 236.2374, 242.3420, 238.4786, 236.7240, 235.3503,
         242.2222, 236.7791, 239.7298, 236.7359, 235.6611, 239.7487, 233.7325,
         241.4185, 232.1245, 236.0488, 232.0436, 238.4375, 233.6498, 229.4159,
         235.1036, 233.7758, 236.7264, 244.0305, 236.5325, 232.6063, 231.7659,
         237.5415, 238.4850, 220.3715, 229.0396, 231.1714, 233.6785, 219.6081,
         229.9293, 226.1010, 232.7872, 233.0021, 237.1839, 238.7932, 236.1073,
         238.1964, 233.6539, 23

Top-5개의 passage를 retrieve 하고 ground truth와 비교하기

In [36]:
k = 5
print("[Search query]\n", query, "\n")
print("[Ground truth passage]")
print(ground_truth, "\n")

for i in range(k):
  print("Top-%d passage with score %.4f" % (i+1, dot_prod_scores.squeeze()[rank[i]]))
  print(valid_corpus[rank[i]])

[Search query]
 주이로부터 문흠의 가짜 항복 소식을 들은 인물은? 

[Ground truth passage]
아버지 주환이 고위 관료였기 때문에, 낭(郞)에 서임되었다. 기도위가 되어, 주환의 병사를 대신 거느렸다\n\n적오 4년(241년), 오나라에서 병사를 크게 일으켜 위나라를 세 방향으로 치고 들어갔을 때, 주연을 따라 위나라의 번성을 공격했고, 여거와 함께 성의 외곽 방어를 깨트렸다. 주연 군은 사마의의 구원군을 보고 달아나다가 대패했으나, 주이는 이 공적으로 돌아와서는 편장군이 되었다\n\n위나라의 여강태수 문흠이 육안에 주둔하면서 수많은 요새를 구축하고, 오나라 사람들에게 모반·망명을 유도하여 오나라에 피해를 입히고 있었다. 주이는 몸소 수하 2천 명을 거느리고 문흠의 일곱 주둔지를 격파하여 수백 급을 베었고, 이 공적으로 양무장군으로 전임했다\n\n적오 13년(250년) 10월 , 당시 양주자사인 문흠의 투항 밀사를 받아 친히 문흠을 맞이해 달라는 청을 받았다. 그러나 이것이 거짓 투항임을 간파하고 대제 손권에게 아뢨으며, 손권이 받아들여 여거에게 2만 명을 거느리고 주이를 도와 함께 국경에서 문흠을 기다리게 했다. 문흠은 결국 투항하지 않았다\n\n건흥 원년(252년)에는 진남장군으로 전임했다. 이해 12월, 위나라에서 대제가 죽은 틈을 노리고 군사를 크게 일으켜 세 방향으로 오나라를 쳤으며, 제갈탄과 호준이 그 중 한 방향을 맡아 지난달에 오나라의 태부 제갈각이 쌓은 동흥의 두 성을 함몰하고자 했다. 주이는 이 싸움에서 수군을 거느리고 위나라의 부교를 쳐 무너뜨려 위나라 군사를 크게 무찔렀다\n\n태평 2년(258년) 6월 , 손침의 명령을 받고 호림(虎林)에서 출발하여 하구독 손일을 암습하려 했는데, 무창에 이르렀을 때 손일이 이를 알아채고 위나라로 달아났다. 7월에 하구에 이르렀다 가절을 받고 전부독(손량전에 따른 것으로, 주환전에서는 대도독)이 되어, 수춘에서 위나라에 반기를 들고 오나라에 도움을 구한 제갈탄을 도와 수춘성의